In [15]:
%%capture
!pip install torch-geometric

In [16]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
from itertools import product

In [17]:
X_train = pd.read_json('arc-agi_training_challenges.json', orient='index')
y_train = pd.read_json('arc-agi_training_solutions.json', orient='index')

In [18]:
''' This function will take the rows of the above X_train, and split
the data into different inputs and outputs of the training example, before
providing the test example'''

def split_data(row):
  inputs = []
  outputs = []
  for element in row:
    inputs.append(element['input'])
    outputs.append(element['output'])
  return pd.Series({'input': inputs, 'output': outputs})


In [19]:
## Seperate input and output into seperate columns
X_train = pd.concat([X_train, X_train['train'].apply(split_data)], axis=1)

In [20]:
X_train = X_train[["input", "output", "test"]]

In [21]:
'''This function will split each input and output for a corresponding
reasoning task, it will also refine the test column'''

def format_data(data):
  new_data = data.explode('input').explode('output')
  new_data['test'] = new_data['test'].apply(lambda x: x[0]['input'])
  return new_data

In [22]:
X_train = format_data(X_train)

In [37]:
def create_data_object(row):
  # Define your 2D array and the corresponding colors array
  colors = np.array(row)
  print(colors)

  array = np.arange(colors.shape[0] * colors.shape[1]).reshape(colors.shape[0],
                                                               colors.shape[1])
  # Function to create edges with diagonal connections
  def create_edges_2d_with_diagonals(array):
      rows = len(array)
      cols = len(array[0])
      edges = []

      for i in range(rows):
          for j in range(cols):
              for di in [-1, 0, 1]:
                  for dj in [-1, 0, 1]:
                      if di == 0 and dj == 0:
                          continue
                      ni, nj = i + di, j + dj
                      if 0 <= ni < rows and 0 <= nj < cols:
                          edges.append((i * cols + j, ni * cols + nj))

      return edges

  # Create edges
  edges = create_edges_2d_with_diagonals(array)

  # Flatten the colors array to create node features
  node_features = colors.flatten().reshape(-1, 1)

  # Convert to PyTorch tensors
  edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
  x = torch.tensor(node_features, dtype=torch.float)

  # Create the PyTorch Geometric Data object
  data = Data(x=x, edge_index=edge_index)

  return data

In [38]:
one_example = create_data_object(X_train['input'][0])

[[0 7 7]
 [7 7 7]
 [0 7 7]]


<ipython-input-38-fd45aa2110d2>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  one_example = create_data_object(X_train['input'][0])
